In [ ]:
import os
import pytrend
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
session = pytrend.itrend_developer_tools()

session.set_credentials(
    access_key_id = '',    # Ingresa tu access_key_id
    secret_access_key = '' # Ingresa tu secret_access_key
)

# Conjunto de datos
dataset_id = 'itrend-ds:ZW5TFERBT8B0GMQ'
collection_id = 'Identificador'

# Obtener formatos disponibles
dataset_formats = session.get_dataset_formats(dataset_id)
fmt = dataset_formats[0] # Escoger el formato que más le acomode: dataset_formats = [csv, geojson, shp]

# Descargar archivo
response = session.download_file(dataset_id, fmt)
print(response)

In [ ]:
# Leemos la tabla
df = pd.read_csv(response.get('filename'), delimiter=response.get('delimiter'))
df.head()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
hist = ax.hist(df['Magnitud [*]'], ec='k', bins=30)
_ = ax.set_ylabel('Número de eventos')
_ = ax.set_xlabel('Magnitud [*]')

In [ ]:
# Descargaremos los registros del evento más reciente
element_id = df.iloc[-1][collection_id]
print(element_id)

In [ ]:
element_formats = session.get_element_formats(dataset_id)
print(element_formats)
fmt = element_formats[1] # npz

In [ ]:
response = session.download_file(dataset_id, fmt, element_id)
print(response)

In [ ]:
with np.load(response['filename'], allow_pickle=True) as f:
    event = {}
    for key, value in f.items():
        event[key] = value.item()

print('Estaciones', list(event.keys()))

In [ ]:
station_id = list(event.keys())[0]
station = event.get(station_id)

channels = list(station.keys())
channel = station.get(channels[0])
print(channels)
print(list(channel.keys()))

In [ ]:
fig = plt.figure(figsize=(12,12))

for i, (channel_id, channel) in enumerate(station.items()):
    ax = fig.add_subplot(3, 1, i+1)
    ax.plot(channel['x'], channel['y'], label=channel_id, lw=0.5)
    ax.set_xlabel(channel['lab']['x'])
    ax.set_ylabel(channel['lab']['y'])
    ax.legend()
    ax.grid(True)
    
_ = fig.suptitle('Evento: ' + df.iloc[-1][collection_id] +
                 '\nFecha: ' + df.iloc[-1]['Fecha (UTC)'] +
                 '\nEstación: ' + station_id)